**OpenTable reservation time scraper**

Link: https://www.opentable.co.uk/s?dateTime=2026-01-31T23%3A30%3A00&covers=2&metroId=3146&shouldUseLatLongSearch=false&originCorrelationId=6b9e96ee-1ae5-4791-a91a-e60d7091faf0&corrid=77e4b75f-cde3-4d45-b7b1-14790e7f372f&queryUnderstandingType=none&showMap=true&sortBy=web_conversion

31st Jan 2026, 23:30, 2px

In [2]:
# load packages
!pip install selenium webdriver-manager beautifulsoup4

import requests
from bs4 import BeautifulSoup
import time
import csv
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

In [ ]:
# aberdeen
import csv
import time
from bs4 import BeautifulSoup
import undetected_chromedriver as uc

URL = "https://www.opentable.co.uk/s?dateTime=2026-01-31T23%3A30%3A00&covers=2&metroId=3029&regionIds%5B%5D=&neighborhoodIds%5B%5D=&term=&shouldUseLatLongSearch=false"

def scrape_opentable_uc_scroll(url):
    print("[1/6] Creating fresh Chrome options...")
    
    options = uc.ChromeOptions()
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--start-maximized")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-gpu")
    options.add_argument("--disable-dev-shm-usage")
    options.headless = False  # make browser visible

    print("[2/6] Launching undetected Chrome...")
    driver = uc.Chrome(options=options, version_main=142)

    try:
        print("[3/6] Loading OpenTable page...")
        driver.get(url)
        time.sleep(3)

        # ----- SCROLL SLOWLY TO LOAD ALL CARDS -----
        print("[4/6] Scrolling page to load all restaurants...")
        last_height = driver.execute_script("return document.body.scrollHeight")
        
        while True:
            driver.execute_script("window.scrollBy(0, 500);")  # scroll down 500px
            time.sleep(1)  # wait for lazy-loaded content
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break  # reached bottom
            last_height = new_height

        print("[5/6] Extracting restaurant data...")
        soup = BeautifulSoup(driver.page_source, "html.parser")
        cards = soup.find_all("div", attrs={"data-test": "restaurant-card"})
        print(f"Found {len(cards)} restaurant cards.\n")

        results = []

        for idx, card in enumerate(cards, start=1):
            print(f" → Processing restaurant {idx}/{len(cards)}")
            try:
                restaurant_id = card.get("data-rid")

                name_tag = card.find("a", attrs={"data-test": "res-card-name"})
                name = name_tag.get_text(strip=True) if name_tag else None

                rating_tag = card.select_one(".tSiVMQB9es0-")
                rating = rating_tag.get_text(strip=True) if rating_tag else None

                review_tag = card.find("a", attrs={"href": lambda x: x and "#reviews" in x})
                reviews = review_tag.get_text(strip=True) if review_tag else None

                price_tag = card.select_one(".Vk-xtpOrXcE-")
                price = price_tag.get_text(strip=True) if price_tag else None

                cuisine_tag = card.select_one('[data-test="cuisine-and-location"]')
                cuisine_location = cuisine_tag.get_text(strip=True) if cuisine_tag else None

                slots = [
                    slot.get_text(strip=True)
                    for slot in card.select('[data-test^="time-slot"] div')
                    if slot.get_text(strip=True)
                ]

                results.append({
                    "restaurant_id": restaurant_id,
                    "name": name,
                    "rating": rating,
                    "reviews": reviews,
                    "price": price,
                    "cuisine_location": cuisine_location,
                    "timeslots": ", ".join(slots),
                })

            except Exception as e:
                print(f"   ! Error processing card {idx}: {e}")

            time.sleep(0.05)

        print("\n[6/6] Saving results to CSV...")
        with open("restaurants.csv", "w", newline="", encoding="utf-8") as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=[
                "restaurant_id", "name", "rating", "reviews",
                "price", "cuisine_location", "timeslots"
            ])
            writer.writeheader()
            writer.writerows(results)

        print("Scraping complete! Saved as restaurants.csv\n")

    finally:
        driver.quit()

    return results


if __name__ == "__main__":
    scrape_opentable_uc_scroll(URL)


In [16]:
# birmingham
URL = "https://www.opentable.co.uk/s?dateTime=2026-01-31T23%3A30%3A00&covers=2&metroId=3146&shouldUseLatLongSearch=false&originCorrelationId=6b9e96ee-1ae5-4791-a91a-e60d7091faf0&corrid=77e4b75f-cde3-4d45-b7b1-14790e7f372f&queryUnderstandingType=none&showMap=true&sortBy=web_conversion"

def scrape_opentable_birmingham_one_click(url):
    print("[1/6] Creating fresh Chrome options...")
    
    options = uc.ChromeOptions()
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--start-maximized")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-gpu")
    options.add_argument("--disable-dev-shm-usage")
    options.headless = False  # make browser visible

    print("[2/6] Launching undetected Chrome...")
    driver = uc.Chrome(options=options, version_main=142)

    all_results = []

    try:
        driver.get(url)
        page_number = 1
        next_clicked = False  # flag to click next page only once

        while True:
            print(f"\n[3/6] Processing page {page_number}...")
            time.sleep(3)

            # ----- SCROLL SLOWLY TO LOAD ALL CARDS -----
            print("[4/6] Scrolling page to load all restaurants...")
            last_height = driver.execute_script("return document.body.scrollHeight")
            
            while True:
                driver.execute_script("window.scrollBy(0, 500);")
                time.sleep(1)
                new_height = driver.execute_script("return document.body.scrollHeight")
                if new_height == last_height:
                    break
                last_height = new_height

            # Parse page
            soup = BeautifulSoup(driver.page_source, "html.parser")
            cards = soup.find_all("div", attrs={"data-test": "restaurant-card"})
            print(f"Found {len(cards)} restaurant cards on page {page_number}.")

            # Extract info
            for idx, card in enumerate(cards, start=1):
                print(f" → Processing restaurant {idx}/{len(cards)}")
                try:
                    restaurant_id = card.get("data-rid")

                    name_tag = card.find("a", attrs={"data-test": "res-card-name"})
                    name = name_tag.get_text(strip=True) if name_tag else None

                    rating_tag = card.select_one(".tSiVMQB9es0-")
                    rating = rating_tag.get_text(strip=True) if rating_tag else None

                    review_tag = card.find("a", attrs={"href": lambda x: x and "#reviews" in x})
                    reviews = review_tag.get_text(strip=True) if review_tag else None

                    price_tag = card.select_one(".Vk-xtpOrXcE-")
                    price = price_tag.get_text(strip=True) if price_tag else None

                    cuisine_tag = card.select_one('[data-test="cuisine-and-location"]')
                    cuisine_location = cuisine_tag.get_text(strip=True) if cuisine_tag else None

                    slots = [
                        slot.get_text(strip=True)
                        for slot in card.select('[data-test^="time-slot"] div')
                        if slot.get_text(strip=True)
                    ]

                    all_results.append({
                        "restaurant_id": restaurant_id,
                        "name": name,
                        "rating": rating,
                        "reviews": reviews,
                        "price": price,
                        "cuisine_location": cuisine_location,
                        "timeslots": ", ".join(slots),
                    })

                except Exception as e:
                    print(f"   ! Error processing card {idx}: {e}")
                time.sleep(0.05)

            # ----- CLICK NEXT PAGE ONCE -----
            if not next_clicked:
                try:
                    next_button = WebDriverWait(driver, 5).until(
                        EC.presence_of_element_located(
                            (By.CSS_SELECTOR, 'a[aria-label="Go to the next page"]')
                        )
                    )
                    driver.execute_script("arguments[0].scrollIntoView(true);", next_button)
                    time.sleep(1)
                    driver.execute_script("arguments[0].click();", next_button)
                    next_clicked = True
                    page_number += 1
                    time.sleep(3)
                except TimeoutException:
                    print("No next page button found, finished scraping.")
                    break
            else:
                break  # already clicked once, stop here

        # ----- SAVE CSV -----
        print("\n[5/6] Saving results to 'birmingham_west_midlands.csv'...")
        with open("birmingham_west_midlands.csv", "w", newline="", encoding="utf-8") as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=[
                "restaurant_id", "name", "rating", "reviews",
                "price", "cuisine_location", "timeslots"
            ])
            writer.writeheader()
            writer.writerows(all_results)

        print(f"[6/6] Scraping complete! Total restaurants: {len(all_results)}\n")

    finally:
        driver.quit()

    return all_results


if __name__ == "__main__":
    scrape_opentable_birmingham_one_click(URL)


[1/6] Creating fresh Chrome options...
[2/6] Launching undetected Chrome...

[3/6] Processing page 1...
[4/6] Scrolling page to load all restaurants...
Found 50 restaurant cards on page 1.
 → Processing restaurant 1/50
 → Processing restaurant 2/50
 → Processing restaurant 3/50
 → Processing restaurant 4/50
 → Processing restaurant 5/50
 → Processing restaurant 6/50
 → Processing restaurant 7/50
 → Processing restaurant 8/50
 → Processing restaurant 9/50
 → Processing restaurant 10/50
 → Processing restaurant 11/50
 → Processing restaurant 12/50
 → Processing restaurant 13/50
 → Processing restaurant 14/50
 → Processing restaurant 15/50
 → Processing restaurant 16/50
 → Processing restaurant 17/50
 → Processing restaurant 18/50
 → Processing restaurant 19/50
 → Processing restaurant 20/50
 → Processing restaurant 21/50
 → Processing restaurant 22/50
 → Processing restaurant 23/50
 → Processing restaurant 24/50
 → Processing restaurant 25/50
 → Processing restaurant 26/50
 → Processing 

In [18]:
# brighton
import csv
import time
from bs4 import BeautifulSoup
import undetected_chromedriver as uc

URL = "https://www.opentable.co.uk/s?dateTime=2026-01-31T23%3A30%3A00&covers=2&metroId=3089&regionIds%5B%5D=&neighborhoodIds%5B%5D=&term=&shouldUseLatLongSearch=false&originCorrelationId=550d3ccc-a24c-4a57-bd6e-05d0487accdf"

def scrape_opentable_uc_scroll(url):
    print("[1/6] Creating fresh Chrome options...")
    
    options = uc.ChromeOptions()
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--start-maximized")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-gpu")
    options.add_argument("--disable-dev-shm-usage")
    options.headless = False  # make browser visible

    print("[2/6] Launching undetected Chrome...")
    driver = uc.Chrome(options=options, version_main=142)

    try:
        print("[3/6] Loading OpenTable page...")
        driver.get(url)
        time.sleep(3)

        # ----- SCROLL SLOWLY TO LOAD ALL CARDS -----
        print("[4/6] Scrolling page to load all restaurants...")
        last_height = driver.execute_script("return document.body.scrollHeight")
        
        while True:
            driver.execute_script("window.scrollBy(0, 500);")  # scroll down 500px
            time.sleep(1)  # wait for lazy-loaded content
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break  # reached bottom
            last_height = new_height

        print("[5/6] Extracting restaurant data...")
        soup = BeautifulSoup(driver.page_source, "html.parser")
        cards = soup.find_all("div", attrs={"data-test": "restaurant-card"})
        print(f"Found {len(cards)} restaurant cards.\n")

        results = []

        for idx, card in enumerate(cards, start=1):
            print(f" → Processing restaurant {idx}/{len(cards)}")
            try:
                restaurant_id = card.get("data-rid")

                name_tag = card.find("a", attrs={"data-test": "res-card-name"})
                name = name_tag.get_text(strip=True) if name_tag else None

                rating_tag = card.select_one(".tSiVMQB9es0-")
                rating = rating_tag.get_text(strip=True) if rating_tag else None

                review_tag = card.find("a", attrs={"href": lambda x: x and "#reviews" in x})
                reviews = review_tag.get_text(strip=True) if review_tag else None

                price_tag = card.select_one(".Vk-xtpOrXcE-")
                price = price_tag.get_text(strip=True) if price_tag else None

                cuisine_tag = card.select_one('[data-test="cuisine-and-location"]')
                cuisine_location = cuisine_tag.get_text(strip=True) if cuisine_tag else None

                slots = [
                    slot.get_text(strip=True)
                    for slot in card.select('[data-test^="time-slot"] div')
                    if slot.get_text(strip=True)
                ]

                results.append({
                    "restaurant_id": restaurant_id,
                    "name": name,
                    "rating": rating,
                    "reviews": reviews,
                    "price": price,
                    "cuisine_location": cuisine_location,
                    "timeslots": ", ".join(slots),
                })

            except Exception as e:
                print(f"   ! Error processing card {idx}: {e}")

            time.sleep(0.05)

        print("\n[6/6] Saving results to CSV...")
        with open("brighton.csv", "w", newline="", encoding="utf-8") as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=[
                "restaurant_id", "name", "rating", "reviews",
                "price", "cuisine_location", "timeslots"
            ])
            writer.writeheader()
            writer.writerows(results)

        print("Scraping complete! Saved as restaurants.csv\n")

    finally:
        driver.quit()

    return results


if __name__ == "__main__":
    scrape_opentable_uc_scroll(URL)

[1/6] Creating fresh Chrome options...
[2/6] Launching undetected Chrome...
[3/6] Loading OpenTable page...
[4/6] Scrolling page to load all restaurants...
[5/6] Extracting restaurant data...
Found 50 restaurant cards.

 → Processing restaurant 1/50
 → Processing restaurant 2/50
 → Processing restaurant 3/50
 → Processing restaurant 4/50
 → Processing restaurant 5/50
 → Processing restaurant 6/50
 → Processing restaurant 7/50
 → Processing restaurant 8/50
 → Processing restaurant 9/50
 → Processing restaurant 10/50
 → Processing restaurant 11/50
 → Processing restaurant 12/50
 → Processing restaurant 13/50
 → Processing restaurant 14/50
 → Processing restaurant 15/50
 → Processing restaurant 16/50
 → Processing restaurant 17/50
 → Processing restaurant 18/50
 → Processing restaurant 19/50
 → Processing restaurant 20/50
 → Processing restaurant 21/50
 → Processing restaurant 22/50
 → Processing restaurant 23/50
 → Processing restaurant 24/50
 → Processing restaurant 25/50
 → Processing 

In [19]:
# edinburgh
URL = "https://www.opentable.co.uk/s?dateTime=2026-01-31T23%3A30%3A00&covers=2&metroId=3191&shouldUseLatLongSearch=false&originCorrelationId=f1b4cbb5-e6d8-4c0b-902b-3bce62b50602&corrid=0f2529b5-29dd-4066-9077-6f2333afc0aa&queryUnderstandingType=none&showMap=true&sortBy=web_conversion"

def scrape_opentable_edinburgh(url):
    print("[1/6] Creating fresh Chrome options...")
    
    options = uc.ChromeOptions()
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--start-maximized")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-gpu")
    options.add_argument("--disable-dev-shm-usage")
    options.headless = False  # make browser visible

    print("[2/6] Launching undetected Chrome...")
    driver = uc.Chrome(options=options, version_main=142)

    all_results = []

    try:
        driver.get(url)
        page_number = 1
        next_clicks_remaining = 3  # click next page 3 times for 4 pages total

        while True:
            print(f"\n[3/6] Processing page {page_number}...")
            time.sleep(3)

            # ----- SCROLL SLOWLY TO LOAD ALL CARDS -----
            print("[4/6] Scrolling page to load all restaurants...")
            last_height = driver.execute_script("return document.body.scrollHeight")
            
            while True:
                driver.execute_script("window.scrollBy(0, 500);")
                time.sleep(1)
                new_height = driver.execute_script("return document.body.scrollHeight")
                if new_height == last_height:
                    break
                last_height = new_height

            # Parse page
            soup = BeautifulSoup(driver.page_source, "html.parser")
            cards = soup.find_all("div", attrs={"data-test": "restaurant-card"})
            print(f"Found {len(cards)} restaurant cards on page {page_number}.")

            # Extract info
            for idx, card in enumerate(cards, start=1):
                print(f" → Processing restaurant {idx}/{len(cards)}")
                try:
                    restaurant_id = card.get("data-rid")

                    name_tag = card.find("a", attrs={"data-test": "res-card-name"})
                    name = name_tag.get_text(strip=True) if name_tag else None

                    rating_tag = card.select_one(".tSiVMQB9es0-")
                    rating = rating_tag.get_text(strip=True) if rating_tag else None

                    review_tag = card.find("a", attrs={"href": lambda x: x and "#reviews" in x})
                    reviews = review_tag.get_text(strip=True) if review_tag else None

                    price_tag = card.select_one(".Vk-xtpOrXcE-")
                    price = price_tag.get_text(strip=True) if price_tag else None

                    cuisine_tag = card.select_one('[data-test="cuisine-and-location"]')
                    cuisine_location = cuisine_tag.get_text(strip=True) if cuisine_tag else None

                    slots = [
                        slot.get_text(strip=True)
                        for slot in card.select('[data-test^="time-slot"] div')
                        if slot.get_text(strip=True)
                    ]

                    all_results.append({
                        "restaurant_id": restaurant_id,
                        "name": name,
                        "rating": rating,
                        "reviews": reviews,
                        "price": price,
                        "cuisine_location": cuisine_location,
                        "timeslots": ", ".join(slots),
                    })

                except Exception as e:
                    print(f"   ! Error processing card {idx}: {e}")
                time.sleep(0.05)

            # ----- CLICK NEXT PAGE UP TO 3 TIMES -----
            if next_clicks_remaining > 0:
                try:
                    next_button = WebDriverWait(driver, 5).until(
                        EC.presence_of_element_located(
                            (By.CSS_SELECTOR, 'a[aria-label="Go to the next page"]')
                        )
                    )
                    driver.execute_script("arguments[0].scrollIntoView(true);", next_button)
                    time.sleep(1)
                    driver.execute_script("arguments[0].click();", next_button)
                    next_clicks_remaining -= 1
                    page_number += 1
                    time.sleep(3)
                except TimeoutException:
                    print("No next page button found, finished scraping.")
                    break
            else:
                break  # clicked 3 times, stop here

        # ----- SAVE CSV -----
        print("\n[5/6] Saving results to 'edinburgh.csv'...")
        with open("edinburgh.csv", "w", newline="", encoding="utf-8") as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=[
                "restaurant_id", "name", "rating", "reviews",
                "price", "cuisine_location", "timeslots"
            ])
            writer.writeheader()
            writer.writerows(all_results)

        print(f"[6/6] Scraping complete! Total restaurants: {len(all_results)}\n")

    finally:
        driver.quit()

    return all_results


if __name__ == "__main__":
    scrape_opentable_edinburgh(URL)


[1/6] Creating fresh Chrome options...
[2/6] Launching undetected Chrome...

[3/6] Processing page 1...
[4/6] Scrolling page to load all restaurants...
Found 50 restaurant cards on page 1.
 → Processing restaurant 1/50
 → Processing restaurant 2/50
 → Processing restaurant 3/50
 → Processing restaurant 4/50
 → Processing restaurant 5/50
 → Processing restaurant 6/50
 → Processing restaurant 7/50
 → Processing restaurant 8/50
 → Processing restaurant 9/50
 → Processing restaurant 10/50
 → Processing restaurant 11/50
 → Processing restaurant 12/50
 → Processing restaurant 13/50
 → Processing restaurant 14/50
 → Processing restaurant 15/50
 → Processing restaurant 16/50
 → Processing restaurant 17/50
 → Processing restaurant 18/50
 → Processing restaurant 19/50
 → Processing restaurant 20/50
 → Processing restaurant 21/50
 → Processing restaurant 22/50
 → Processing restaurant 23/50
 → Processing restaurant 24/50
 → Processing restaurant 25/50
 → Processing restaurant 26/50
 → Processing 

In [20]:
# essex
URL = "https://www.opentable.co.uk/s?dateTime=2026-01-31T23%3A30%3A00&covers=2&metroId=3125&regionIds%5B%5D=&neighborhoodIds%5B%5D=&term=&shouldUseLatLongSearch=false&originCorrelationId=2f122bc1-0b38-45c8-9a50-126f2d596f96"

def scrape_opentable_essex(url):
    print("[1/6] Creating fresh Chrome options...")
    
    options = uc.ChromeOptions()
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--start-maximized")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-gpu")
    options.add_argument("--disable-dev-shm-usage")
    options.headless = False  # make browser visible

    print("[2/6] Launching undetected Chrome...")
    driver = uc.Chrome(options=options, version_main=142)

    all_results = []

    try:
        driver.get(url)
        page_number = 1
        next_clicks_remaining = 1  # only click once

        while True:
            print(f"\n[3/6] Processing page {page_number}...")
            time.sleep(3)

            # ----- SCROLL SLOWLY TO LOAD ALL CARDS -----
            print("[4/6] Scrolling page to load all restaurants...")
            last_height = driver.execute_script("return document.body.scrollHeight")
            
            while True:
                driver.execute_script("window.scrollBy(0, 500);")
                time.sleep(1)
                new_height = driver.execute_script("return document.body.scrollHeight")
                if new_height == last_height:
                    break
                last_height = new_height

            # Parse page
            soup = BeautifulSoup(driver.page_source, "html.parser")
            cards = soup.find_all("div", attrs={"data-test": "restaurant-card"})
            print(f"Found {len(cards)} restaurant cards on page {page_number}.")

            # Extract info
            for idx, card in enumerate(cards, start=1):
                print(f" → Processing restaurant {idx}/{len(cards)}")
                try:
                    restaurant_id = card.get("data-rid")

                    name_tag = card.find("a", attrs={"data-test": "res-card-name"})
                    name = name_tag.get_text(strip=True) if name_tag else None

                    rating_tag = card.select_one(".tSiVMQB9es0-")
                    rating = rating_tag.get_text(strip=True) if rating_tag else None

                    review_tag = card.find("a", attrs={"href": lambda x: x and "#reviews" in x})
                    reviews = review_tag.get_text(strip=True) if review_tag else None

                    price_tag = card.select_one(".Vk-xtpOrXcE-")
                    price = price_tag.get_text(strip=True) if price_tag else None

                    cuisine_tag = card.select_one('[data-test="cuisine-and-location"]')
                    cuisine_location = cuisine_tag.get_text(strip=True) if cuisine_tag else None

                    slots = [
                        slot.get_text(strip=True)
                        for slot in card.select('[data-test^="time-slot"] div')
                        if slot.get_text(strip=True)
                    ]

                    all_results.append({
                        "restaurant_id": restaurant_id,
                        "name": name,
                        "rating": rating,
                        "reviews": reviews,
                        "price": price,
                        "cuisine_location": cuisine_location,
                        "timeslots": ", ".join(slots),
                    })

                except Exception as e:
                    print(f"   ! Error processing card {idx}: {e}")
                time.sleep(0.05)

            # ----- CLICK NEXT PAGE ONCE -----
            if next_clicks_remaining > 0:
                try:
                    next_button = WebDriverWait(driver, 5).until(
                        EC.presence_of_element_located(
                            (By.CSS_SELECTOR, 'a[aria-label="Go to the next page"]')
                        )
                    )
                    driver.execute_script("arguments[0].scrollIntoView(true);", next_button)
                    time.sleep(1)
                    driver.execute_script("arguments[0].click();", next_button)
                    next_clicks_remaining -= 1
                    page_number += 1
                    time.sleep(3)
                except TimeoutException:
                    print("No next page button found, finished scraping.")
                    break
            else:
                break  # clicked once, stop here

        # ----- SAVE CSV -----
        print("\n[5/6] Saving results to 'essex.csv'...")
        with open("essex.csv", "w", newline="", encoding="utf-8") as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=[
                "restaurant_id", "name", "rating", "reviews",
                "price", "cuisine_location", "timeslots"
            ])
            writer.writeheader()
            writer.writerows(all_results)

        print(f"[6/6] Scraping complete! Total restaurants: {len(all_results)}\n")

    finally:
        driver.quit()

    return all_results


if __name__ == "__main__":
    scrape_opentable_essex(URL)


[1/6] Creating fresh Chrome options...
[2/6] Launching undetected Chrome...

[3/6] Processing page 1...
[4/6] Scrolling page to load all restaurants...
Found 50 restaurant cards on page 1.
 → Processing restaurant 1/50
 → Processing restaurant 2/50
 → Processing restaurant 3/50
 → Processing restaurant 4/50
 → Processing restaurant 5/50
 → Processing restaurant 6/50
 → Processing restaurant 7/50
 → Processing restaurant 8/50
 → Processing restaurant 9/50
 → Processing restaurant 10/50
 → Processing restaurant 11/50
 → Processing restaurant 12/50
 → Processing restaurant 13/50
 → Processing restaurant 14/50
 → Processing restaurant 15/50
 → Processing restaurant 16/50
 → Processing restaurant 17/50
 → Processing restaurant 18/50
 → Processing restaurant 19/50
 → Processing restaurant 20/50
 → Processing restaurant 21/50
 → Processing restaurant 22/50
 → Processing restaurant 23/50
 → Processing restaurant 24/50
 → Processing restaurant 25/50
 → Processing restaurant 26/50
 → Processing 

In [21]:
# glasgow
URL = "https://www.opentable.co.uk/s?dateTime=2026-01-31T23%3A30%3A00&covers=2&metroId=3188&regionIds%5B%5D=&neighborhoodIds%5B%5D=&term=&shouldUseLatLongSearch=false&originCorrelationId=5c236a4f-f291-4199-8d33-6945dbf4ce9e"

def scrape_opentable_glasgow(url):
    print("[1/6] Creating fresh Chrome options...")
    
    options = uc.ChromeOptions()
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--start-maximized")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-gpu")
    options.add_argument("--disable-dev-shm-usage")
    options.headless = False  # make browser visible

    print("[2/6] Launching undetected Chrome...")
    driver = uc.Chrome(options=options, version_main=142)

    all_results = []

    try:
        driver.get(url)
        page_number = 1
        next_clicks_remaining = 1  # only click once

        while True:
            print(f"\n[3/6] Processing page {page_number}...")
            time.sleep(3)

            # ----- SCROLL SLOWLY TO LOAD ALL CARDS -----
            print("[4/6] Scrolling page to load all restaurants...")
            last_height = driver.execute_script("return document.body.scrollHeight")
            
            while True:
                driver.execute_script("window.scrollBy(0, 500);")
                time.sleep(1)
                new_height = driver.execute_script("return document.body.scrollHeight")
                if new_height == last_height:
                    break
                last_height = new_height

            # Parse page
            soup = BeautifulSoup(driver.page_source, "html.parser")
            cards = soup.find_all("div", attrs={"data-test": "restaurant-card"})
            print(f"Found {len(cards)} restaurant cards on page {page_number}.")

            # Extract info
            for idx, card in enumerate(cards, start=1):
                print(f" → Processing restaurant {idx}/{len(cards)}")
                try:
                    restaurant_id = card.get("data-rid")

                    name_tag = card.find("a", attrs={"data-test": "res-card-name"})
                    name = name_tag.get_text(strip=True) if name_tag else None

                    rating_tag = card.select_one(".tSiVMQB9es0-")
                    rating = rating_tag.get_text(strip=True) if rating_tag else None

                    review_tag = card.find("a", attrs={"href": lambda x: x and "#reviews" in x})
                    reviews = review_tag.get_text(strip=True) if review_tag else None

                    price_tag = card.select_one(".Vk-xtpOrXcE-")
                    price = price_tag.get_text(strip=True) if price_tag else None

                    cuisine_tag = card.select_one('[data-test="cuisine-and-location"]')
                    cuisine_location = cuisine_tag.get_text(strip=True) if cuisine_tag else None

                    slots = [
                        slot.get_text(strip=True)
                        for slot in card.select('[data-test^="time-slot"] div')
                        if slot.get_text(strip=True)
                    ]

                    all_results.append({
                        "restaurant_id": restaurant_id,
                        "name": name,
                        "rating": rating,
                        "reviews": reviews,
                        "price": price,
                        "cuisine_location": cuisine_location,
                        "timeslots": ", ".join(slots),
                    })

                except Exception as e:
                    print(f"   ! Error processing card {idx}: {e}")
                time.sleep(0.05)

            # ----- CLICK NEXT PAGE ONCE -----
            if next_clicks_remaining > 0:
                try:
                    next_button = WebDriverWait(driver, 5).until(
                        EC.presence_of_element_located(
                            (By.CSS_SELECTOR, 'a[aria-label="Go to the next page"]')
                        )
                    )
                    driver.execute_script("arguments[0].scrollIntoView(true);", next_button)
                    time.sleep(1)
                    driver.execute_script("arguments[0].click();", next_button)
                    next_clicks_remaining -= 1
                    page_number += 1
                    time.sleep(3)
                except TimeoutException:
                    print("No next page button found, finished scraping.")
                    break
            else:
                break  # clicked once, stop here

        # ----- SAVE CSV -----
        print("\n[5/6] Saving results to 'glasgow.csv'...")
        with open("glasgow.csv", "w", newline="", encoding="utf-8") as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=[
                "restaurant_id", "name", "rating", "reviews",
                "price", "cuisine_location", "timeslots"
            ])
            writer.writeheader()
            writer.writerows(all_results)

        print(f"[6/6] Scraping complete! Total restaurants: {len(all_results)}\n")

    finally:
        driver.quit()

    return all_results


if __name__ == "__main__":
    scrape_opentable_glasgow(URL)


[1/6] Creating fresh Chrome options...
[2/6] Launching undetected Chrome...

[3/6] Processing page 1...
[4/6] Scrolling page to load all restaurants...
Found 50 restaurant cards on page 1.
 → Processing restaurant 1/50
 → Processing restaurant 2/50
 → Processing restaurant 3/50
 → Processing restaurant 4/50
 → Processing restaurant 5/50
 → Processing restaurant 6/50
 → Processing restaurant 7/50
 → Processing restaurant 8/50
 → Processing restaurant 9/50
 → Processing restaurant 10/50
 → Processing restaurant 11/50
 → Processing restaurant 12/50
 → Processing restaurant 13/50
 → Processing restaurant 14/50
 → Processing restaurant 15/50
 → Processing restaurant 16/50
 → Processing restaurant 17/50
 → Processing restaurant 18/50
 → Processing restaurant 19/50
 → Processing restaurant 20/50
 → Processing restaurant 21/50
 → Processing restaurant 22/50
 → Processing restaurant 23/50
 → Processing restaurant 24/50
 → Processing restaurant 25/50
 → Processing restaurant 26/50
 → Processing 

In [22]:
# gloucestershire
URL = "https://www.opentable.co.uk/s?dateTime=2026-01-31T23%3A30%3A00&covers=2&metroId=3185&regionIds%5B%5D=&neighborhoodIds%5B%5D=&term=&shouldUseLatLongSearch=false&originCorrelationId=0c1698da-32be-4844-82f5-3d9dc818ba40"

def scrape_opentable_gloucestershire(url):
    print("[1/6] Creating fresh Chrome options...")
    
    options = uc.ChromeOptions()
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--start-maximized")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-gpu")
    options.add_argument("--disable-dev-shm-usage")
    options.headless = False  # make browser visible

    print("[2/6] Launching undetected Chrome...")
    driver = uc.Chrome(options=options, version_main=142)

    all_results = []

    try:
        driver.get(url)
        page_number = 1
        next_clicks_remaining = 1  # only click once

        while True:
            print(f"\n[3/6] Processing page {page_number}...")
            time.sleep(3)

            # ----- SCROLL SLOWLY TO LOAD ALL CARDS -----
            print("[4/6] Scrolling page to load all restaurants...")
            last_height = driver.execute_script("return document.body.scrollHeight")
            
            while True:
                driver.execute_script("window.scrollBy(0, 500);")
                time.sleep(1)
                new_height = driver.execute_script("return document.body.scrollHeight")
                if new_height == last_height:
                    break
                last_height = new_height

            # Parse page
            soup = BeautifulSoup(driver.page_source, "html.parser")
            cards = soup.find_all("div", attrs={"data-test": "restaurant-card"})
            print(f"Found {len(cards)} restaurant cards on page {page_number}.")

            # Extract info
            for idx, card in enumerate(cards, start=1):
                print(f" → Processing restaurant {idx}/{len(cards)}")
                try:
                    restaurant_id = card.get("data-rid")

                    name_tag = card.find("a", attrs={"data-test": "res-card-name"})
                    name = name_tag.get_text(strip=True) if name_tag else None

                    rating_tag = card.select_one(".tSiVMQB9es0-")
                    rating = rating_tag.get_text(strip=True) if rating_tag else None

                    review_tag = card.find("a", attrs={"href": lambda x: x and "#reviews" in x})
                    reviews = review_tag.get_text(strip=True) if review_tag else None

                    price_tag = card.select_one(".Vk-xtpOrXcE-")
                    price = price_tag.get_text(strip=True) if price_tag else None

                    cuisine_tag = card.select_one('[data-test="cuisine-and-location"]')
                    cuisine_location = cuisine_tag.get_text(strip=True) if cuisine_tag else None

                    slots = [
                        slot.get_text(strip=True)
                        for slot in card.select('[data-test^="time-slot"] div')
                        if slot.get_text(strip=True)
                    ]

                    all_results.append({
                        "restaurant_id": restaurant_id,
                        "name": name,
                        "rating": rating,
                        "reviews": reviews,
                        "price": price,
                        "cuisine_location": cuisine_location,
                        "timeslots": ", ".join(slots),
                    })

                except Exception as e:
                    print(f"   ! Error processing card {idx}: {e}")
                time.sleep(0.05)

            # ----- CLICK NEXT PAGE ONCE -----
            if next_clicks_remaining > 0:
                try:
                    next_button = WebDriverWait(driver, 5).until(
                        EC.presence_of_element_located(
                            (By.CSS_SELECTOR, 'a[aria-label="Go to the next page"]')
                        )
                    )
                    driver.execute_script("arguments[0].scrollIntoView(true);", next_button)
                    time.sleep(1)
                    driver.execute_script("arguments[0].click();", next_button)
                    next_clicks_remaining -= 1
                    page_number += 1
                    time.sleep(3)
                except TimeoutException:
                    print("No next page button found, finished scraping.")
                    break
            else:
                break  # clicked once, stop here

        # ----- SAVE CSV -----
        print("\n[5/6] Saving results to 'gloucestershire.csv'...")
        with open("gloucestershire.csv", "w", newline="", encoding="utf-8") as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=[
                "restaurant_id", "name", "rating", "reviews",
                "price", "cuisine_location", "timeslots"
            ])
            writer.writeheader()
            writer.writerows(all_results)

        print(f"[6/6] Scraping complete! Total restaurants: {len(all_results)}\n")

    finally:
        driver.quit()

    return all_results


if __name__ == "__main__":
    scrape_opentable_gloucestershire(URL)


[1/6] Creating fresh Chrome options...
[2/6] Launching undetected Chrome...

[3/6] Processing page 1...
[4/6] Scrolling page to load all restaurants...
Found 50 restaurant cards on page 1.
 → Processing restaurant 1/50
 → Processing restaurant 2/50
 → Processing restaurant 3/50
 → Processing restaurant 4/50
 → Processing restaurant 5/50
 → Processing restaurant 6/50
 → Processing restaurant 7/50
 → Processing restaurant 8/50
 → Processing restaurant 9/50
 → Processing restaurant 10/50
 → Processing restaurant 11/50
 → Processing restaurant 12/50
 → Processing restaurant 13/50
 → Processing restaurant 14/50
 → Processing restaurant 15/50
 → Processing restaurant 16/50
 → Processing restaurant 17/50
 → Processing restaurant 18/50
 → Processing restaurant 19/50
 → Processing restaurant 20/50
 → Processing restaurant 21/50
 → Processing restaurant 22/50
 → Processing restaurant 23/50
 → Processing restaurant 24/50
 → Processing restaurant 25/50
 → Processing restaurant 26/50
 → Processing 

In [24]:
# hampshire
URL = "https://www.opentable.co.uk/s?dateTime=2026-01-31T23%3A30%3A00&covers=2&metroId=3092&regionIds%5B%5D=&neighborhoodIds%5B%5D=&term=&shouldUseLatLongSearch=false&originCorrelationId=937a2167-7929-4f1f-9031-8c4da62da22a"

def scrape_opentable_hampshire(url):
    print("[1/6] Creating fresh Chrome options...")
    
    options = uc.ChromeOptions()
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--start-maximized")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-gpu")
    options.add_argument("--disable-dev-shm-usage")
    options.headless = False  # make browser visible

    print("[2/6] Launching undetected Chrome...")
    driver = uc.Chrome(options=options, version_main=142)

    all_results = []

    try:
        driver.get(url)
        page_number = 1
        next_clicks_remaining = 1  # only click once

        while True:
            print(f"\n[3/6] Processing page {page_number}...")
            time.sleep(3)

            # ----- SCROLL SLOWLY TO LOAD ALL CARDS -----
            print("[4/6] Scrolling page to load all restaurants...")
            last_height = driver.execute_script("return document.body.scrollHeight")
            
            while True:
                driver.execute_script("window.scrollBy(0, 500);")
                time.sleep(1)
                new_height = driver.execute_script("return document.body.scrollHeight")
                if new_height == last_height:
                    break
                last_height = new_height

            # Parse page
            soup = BeautifulSoup(driver.page_source, "html.parser")
            cards = soup.find_all("div", attrs={"data-test": "restaurant-card"})
            print(f"Found {len(cards)} restaurant cards on page {page_number}.")

            # Extract info
            for idx, card in enumerate(cards, start=1):
                print(f" → Processing restaurant {idx}/{len(cards)}")
                try:
                    restaurant_id = card.get("data-rid")

                    name_tag = card.find("a", attrs={"data-test": "res-card-name"})
                    name = name_tag.get_text(strip=True) if name_tag else None

                    rating_tag = card.select_one(".tSiVMQB9es0-")
                    rating = rating_tag.get_text(strip=True) if rating_tag else None

                    review_tag = card.find("a", attrs={"href": lambda x: x and "#reviews" in x})
                    reviews = review_tag.get_text(strip=True) if review_tag else None

                    price_tag = card.select_one(".Vk-xtpOrXcE-")
                    price = price_tag.get_text(strip=True) if price_tag else None

                    cuisine_tag = card.select_one('[data-test="cuisine-and-location"]')
                    cuisine_location = cuisine_tag.get_text(strip=True) if cuisine_tag else None

                    slots = [
                        slot.get_text(strip=True)
                        for slot in card.select('[data-test^="time-slot"] div')
                        if slot.get_text(strip=True)
                    ]

                    all_results.append({
                        "restaurant_id": restaurant_id,
                        "name": name,
                        "rating": rating,
                        "reviews": reviews,
                        "price": price,
                        "cuisine_location": cuisine_location,
                        "timeslots": ", ".join(slots),
                    })

                except Exception as e:
                    print(f"   ! Error processing card {idx}: {e}")
                time.sleep(0.05)

            # ----- CLICK NEXT PAGE ONCE -----
            if next_clicks_remaining > 0:
                try:
                    next_button = WebDriverWait(driver, 5).until(
                        EC.presence_of_element_located(
                            (By.CSS_SELECTOR, 'a[aria-label="Go to the next page"]')
                        )
                    )
                    driver.execute_script("arguments[0].scrollIntoView(true);", next_button)
                    time.sleep(1)
                    driver.execute_script("arguments[0].click();", next_button)
                    next_clicks_remaining -= 1
                    page_number += 1
                    time.sleep(3)
                except TimeoutException:
                    print("No next page button found, finished scraping.")
                    break
            else:
                break  # clicked once, stop here

        # ----- SAVE CSV -----
        print("\n[5/6] Saving results to 'hampshire.csv'...")
        with open("hampshire.csv", "w", newline="", encoding="utf-8") as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=[
                "restaurant_id", "name", "rating", "reviews",
                "price", "cuisine_location", "timeslots"
            ])
            writer.writeheader()
            writer.writerows(all_results)

        print(f"[6/6] Scraping complete! Total restaurants: {len(all_results)}\n")

    finally:
        driver.quit()

    return all_results


if __name__ == "__main__":
    scrape_opentable_hampshire(URL)


[1/6] Creating fresh Chrome options...
[2/6] Launching undetected Chrome...

[3/6] Processing page 1...
[4/6] Scrolling page to load all restaurants...
Found 50 restaurant cards on page 1.
 → Processing restaurant 1/50
 → Processing restaurant 2/50
 → Processing restaurant 3/50
 → Processing restaurant 4/50
 → Processing restaurant 5/50
 → Processing restaurant 6/50
 → Processing restaurant 7/50
 → Processing restaurant 8/50
 → Processing restaurant 9/50
 → Processing restaurant 10/50
 → Processing restaurant 11/50
 → Processing restaurant 12/50
 → Processing restaurant 13/50
 → Processing restaurant 14/50
 → Processing restaurant 15/50
 → Processing restaurant 16/50
 → Processing restaurant 17/50
 → Processing restaurant 18/50
 → Processing restaurant 19/50
 → Processing restaurant 20/50
 → Processing restaurant 21/50
 → Processing restaurant 22/50
 → Processing restaurant 23/50
 → Processing restaurant 24/50
 → Processing restaurant 25/50
 → Processing restaurant 26/50
 → Processing 

In [25]:
# hertfordshire
URL = "https://www.opentable.co.uk/s?dateTime=2026-01-31T23%3A30%3A00&covers=2&metroId=3128&regionIds%5B%5D=&neighborhoodIds%5B%5D=&term=&shouldUseLatLongSearch=false&originCorrelationId=5576cd00-921e-4e3f-84e7-3976d5efe639"

def scrape_opentable_hertfordshire(url):
    print("[1/6] Creating fresh Chrome options...")
    
    options = uc.ChromeOptions()
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--start-maximized")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-gpu")
    options.add_argument("--disable-dev-shm-usage")
    options.headless = False  # make browser visible

    print("[2/6] Launching undetected Chrome...")
    driver = uc.Chrome(options=options, version_main=142)

    all_results = []

    try:
        driver.get(url)
        page_number = 1
        next_clicks_remaining = 1  # only click once

        while True:
            print(f"\n[3/6] Processing page {page_number}...")
            time.sleep(3)

            # ----- SCROLL SLOWLY TO LOAD ALL CARDS -----
            print("[4/6] Scrolling page to load all restaurants...")
            last_height = driver.execute_script("return document.body.scrollHeight")
            
            while True:
                driver.execute_script("window.scrollBy(0, 500);")
                time.sleep(1)
                new_height = driver.execute_script("return document.body.scrollHeight")
                if new_height == last_height:
                    break
                last_height = new_height

            # Parse page
            soup = BeautifulSoup(driver.page_source, "html.parser")
            cards = soup.find_all("div", attrs={"data-test": "restaurant-card"})
            print(f"Found {len(cards)} restaurant cards on page {page_number}.")

            # Extract info
            for idx, card in enumerate(cards, start=1):
                print(f" → Processing restaurant {idx}/{len(cards)}")
                try:
                    restaurant_id = card.get("data-rid")

                    name_tag = card.find("a", attrs={"data-test": "res-card-name"})
                    name = name_tag.get_text(strip=True) if name_tag else None

                    rating_tag = card.select_one(".tSiVMQB9es0-")
                    rating = rating_tag.get_text(strip=True) if rating_tag else None

                    review_tag = card.find("a", attrs={"href": lambda x: x and "#reviews" in x})
                    reviews = review_tag.get_text(strip=True) if review_tag else None

                    price_tag = card.select_one(".Vk-xtpOrXcE-")
                    price = price_tag.get_text(strip=True) if price_tag else None

                    cuisine_tag = card.select_one('[data-test="cuisine-and-location"]')
                    cuisine_location = cuisine_tag.get_text(strip=True) if cuisine_tag else None

                    slots = [
                        slot.get_text(strip=True)
                        for slot in card.select('[data-test^="time-slot"] div')
                        if slot.get_text(strip=True)
                    ]

                    all_results.append({
                        "restaurant_id": restaurant_id,
                        "name": name,
                        "rating": rating,
                        "reviews": reviews,
                        "price": price,
                        "cuisine_location": cuisine_location,
                        "timeslots": ", ".join(slots),
                    })

                except Exception as e:
                    print(f"   ! Error processing card {idx}: {e}")
                time.sleep(0.05)

            # ----- CLICK NEXT PAGE ONCE -----
            if next_clicks_remaining > 0:
                try:
                    next_button = WebDriverWait(driver, 5).until(
                        EC.presence_of_element_located(
                            (By.CSS_SELECTOR, 'a[aria-label="Go to the next page"]')
                        )
                    )
                    driver.execute_script("arguments[0].scrollIntoView(true);", next_button)
                    time.sleep(1)
                    driver.execute_script("arguments[0].click();", next_button)
                    next_clicks_remaining -= 1
                    page_number += 1
                    time.sleep(3)
                except TimeoutException:
                    print("No next page button found, finished scraping.")
                    break
            else:
                break  # clicked once, stop here

        # ----- SAVE CSV -----
        print("\n[5/6] Saving results to 'hertfordshire.csv'...")
        with open("hertfordshire.csv", "w", newline="", encoding="utf-8") as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=[
                "restaurant_id", "name", "rating", "reviews",
                "price", "cuisine_location", "timeslots"
            ])
            writer.writeheader()
            writer.writerows(all_results)

        print(f"[6/6] Scraping complete! Total restaurants: {len(all_results)}\n")

    finally:
        driver.quit()

    return all_results


if __name__ == "__main__":
    scrape_opentable_hertfordshire(URL)


[1/6] Creating fresh Chrome options...
[2/6] Launching undetected Chrome...

[3/6] Processing page 1...
[4/6] Scrolling page to load all restaurants...
Found 50 restaurant cards on page 1.
 → Processing restaurant 1/50
 → Processing restaurant 2/50
 → Processing restaurant 3/50
 → Processing restaurant 4/50
 → Processing restaurant 5/50
 → Processing restaurant 6/50
 → Processing restaurant 7/50
 → Processing restaurant 8/50
 → Processing restaurant 9/50
 → Processing restaurant 10/50
 → Processing restaurant 11/50
 → Processing restaurant 12/50
 → Processing restaurant 13/50
 → Processing restaurant 14/50
 → Processing restaurant 15/50
 → Processing restaurant 16/50
 → Processing restaurant 17/50
 → Processing restaurant 18/50
 → Processing restaurant 19/50
 → Processing restaurant 20/50
 → Processing restaurant 21/50
 → Processing restaurant 22/50
 → Processing restaurant 23/50
 → Processing restaurant 24/50
 → Processing restaurant 25/50
 → Processing restaurant 26/50
 → Processing 

In [26]:
# kent
URL = "https://www.opentable.co.uk/s?dateTime=2026-01-31T23%3A30%3A00&covers=2&metroId=3098&regionIds%5B%5D=&neighborhoodIds%5B%5D=&term=&shouldUseLatLongSearch=false&originCorrelationId=d0d4ad73-ecca-4eb8-9098-847ad558744c"

def scrape_opentable_kent(url):
    print("[1/6] Creating fresh Chrome options...")
    
    options = uc.ChromeOptions()
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--start-maximized")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-gpu")
    options.add_argument("--disable-dev-shm-usage")
    options.headless = False  # make browser visible

    print("[2/6] Launching undetected Chrome...")
    driver = uc.Chrome(options=options, version_main=142)

    all_results = []

    try:
        driver.get(url)
        page_number = 1
        next_clicks_remaining = 3  # click 3 times for 4 pages total

        while True:
            print(f"\n[3/6] Processing page {page_number}...")
            time.sleep(3)

            # ----- SCROLL SLOWLY TO LOAD ALL CARDS -----
            print("[4/6] Scrolling page to load all restaurants...")
            last_height = driver.execute_script("return document.body.scrollHeight")
            
            while True:
                driver.execute_script("window.scrollBy(0, 500);")
                time.sleep(1)
                new_height = driver.execute_script("return document.body.scrollHeight")
                if new_height == last_height:
                    break
                last_height = new_height

            # Parse page
            soup = BeautifulSoup(driver.page_source, "html.parser")
            cards = soup.find_all("div", attrs={"data-test": "restaurant-card"})
            print(f"Found {len(cards)} restaurant cards on page {page_number}.")

            # Extract info
            for idx, card in enumerate(cards, start=1):
                print(f" → Processing restaurant {idx}/{len(cards)}")
                try:
                    restaurant_id = card.get("data-rid")

                    name_tag = card.find("a", attrs={"data-test": "res-card-name"})
                    name = name_tag.get_text(strip=True) if name_tag else None

                    rating_tag = card.select_one(".tSiVMQB9es0-")
                    rating = rating_tag.get_text(strip=True) if rating_tag else None

                    review_tag = card.find("a", attrs={"href": lambda x: x and "#reviews" in x})
                    reviews = review_tag.get_text(strip=True) if review_tag else None

                    price_tag = card.select_one(".Vk-xtpOrXcE-")
                    price = price_tag.get_text(strip=True) if price_tag else None

                    cuisine_tag = card.select_one('[data-test="cuisine-and-location"]')
                    cuisine_location = cuisine_tag.get_text(strip=True) if cuisine_tag else None

                    slots = [
                        slot.get_text(strip=True)
                        for slot in card.select('[data-test^="time-slot"] div')
                        if slot.get_text(strip=True)
                    ]

                    all_results.append({
                        "restaurant_id": restaurant_id,
                        "name": name,
                        "rating": rating,
                        "reviews": reviews,
                        "price": price,
                        "cuisine_location": cuisine_location,
                        "timeslots": ", ".join(slots),
                    })

                except Exception as e:
                    print(f"   ! Error processing card {idx}: {e}")
                time.sleep(0.05)

            # ----- CLICK NEXT PAGE -----
            if next_clicks_remaining > 0:
                try:
                    next_button = WebDriverWait(driver, 5).until(
                        EC.presence_of_element_located(
                            (By.CSS_SELECTOR, 'a[aria-label="Go to the next page"]')
                        )
                    )
                    driver.execute_script("arguments[0].scrollIntoView(true);", next_button)
                    time.sleep(1)
                    driver.execute_script("arguments[0].click();", next_button)
                    next_clicks_remaining -= 1
                    page_number += 1
                    time.sleep(3)
                except TimeoutException:
                    print("No next page button found, finished scraping.")
                    break
            else:
                break  # clicked 3 times, stop here

        # ----- SAVE CSV -----
        print("\n[5/6] Saving results to 'kent.csv'...")
        with open("kent.csv", "w", newline="", encoding="utf-8") as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=[
                "restaurant_id", "name", "rating", "reviews",
                "price", "cuisine_location", "timeslots"
            ])
            writer.writeheader()
            writer.writerows(all_results)

        print(f"[6/6] Scraping complete! Total restaurants: {len(all_results)}\n")

    finally:
        driver.quit()

    return all_results


if __name__ == "__main__":
    scrape_opentable_kent(URL)


[1/6] Creating fresh Chrome options...
[2/6] Launching undetected Chrome...

[3/6] Processing page 1...
[4/6] Scrolling page to load all restaurants...
Found 50 restaurant cards on page 1.
 → Processing restaurant 1/50
 → Processing restaurant 2/50
 → Processing restaurant 3/50
 → Processing restaurant 4/50
 → Processing restaurant 5/50
 → Processing restaurant 6/50
 → Processing restaurant 7/50
 → Processing restaurant 8/50
 → Processing restaurant 9/50
 → Processing restaurant 10/50
 → Processing restaurant 11/50
 → Processing restaurant 12/50
 → Processing restaurant 13/50
 → Processing restaurant 14/50
 → Processing restaurant 15/50
 → Processing restaurant 16/50
 → Processing restaurant 17/50
 → Processing restaurant 18/50
 → Processing restaurant 19/50
 → Processing restaurant 20/50
 → Processing restaurant 21/50
 → Processing restaurant 22/50
 → Processing restaurant 23/50
 → Processing restaurant 24/50
 → Processing restaurant 25/50
 → Processing restaurant 26/50
 → Processing 

In [27]:
# liverpool
URL = "https://www.opentable.co.uk/s?dateTime=2026-01-31T23%3A30%3A00&covers=2&metroId=3176&regionIds%5B%5D=&neighborhoodIds%5B%5D=&term=&shouldUseLatLongSearch=false&originCorrelationId=004ce382-43c3-4c64-bccc-4ab5d52c374c"
def scrape_opentable_uc_scroll(url):
    print("[1/6] Creating fresh Chrome options...")
    
    options = uc.ChromeOptions()
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--start-maximized")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-gpu")
    options.add_argument("--disable-dev-shm-usage")
    options.headless = False  # make browser visible

    print("[2/6] Launching undetected Chrome...")
    driver = uc.Chrome(options=options, version_main=142)

    try:
        print("[3/6] Loading OpenTable page...")
        driver.get(url)
        time.sleep(3)

        # ----- SCROLL SLOWLY TO LOAD ALL CARDS -----
        print("[4/6] Scrolling page to load all restaurants...")
        last_height = driver.execute_script("return document.body.scrollHeight")
        
        while True:
            driver.execute_script("window.scrollBy(0, 500);")  # scroll down 500px
            time.sleep(1)  # wait for lazy-loaded content
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break  # reached bottom
            last_height = new_height

        print("[5/6] Extracting restaurant data...")
        soup = BeautifulSoup(driver.page_source, "html.parser")
        cards = soup.find_all("div", attrs={"data-test": "restaurant-card"})
        print(f"Found {len(cards)} restaurant cards.\n")

        results = []

        for idx, card in enumerate(cards, start=1):
            print(f" → Processing restaurant {idx}/{len(cards)}")
            try:
                restaurant_id = card.get("data-rid")

                name_tag = card.find("a", attrs={"data-test": "res-card-name"})
                name = name_tag.get_text(strip=True) if name_tag else None

                rating_tag = card.select_one(".tSiVMQB9es0-")
                rating = rating_tag.get_text(strip=True) if rating_tag else None

                review_tag = card.find("a", attrs={"href": lambda x: x and "#reviews" in x})
                reviews = review_tag.get_text(strip=True) if review_tag else None

                price_tag = card.select_one(".Vk-xtpOrXcE-")
                price = price_tag.get_text(strip=True) if price_tag else None

                cuisine_tag = card.select_one('[data-test="cuisine-and-location"]')
                cuisine_location = cuisine_tag.get_text(strip=True) if cuisine_tag else None

                slots = [
                    slot.get_text(strip=True)
                    for slot in card.select('[data-test^="time-slot"] div')
                    if slot.get_text(strip=True)
                ]

                results.append({
                    "restaurant_id": restaurant_id,
                    "name": name,
                    "rating": rating,
                    "reviews": reviews,
                    "price": price,
                    "cuisine_location": cuisine_location,
                    "timeslots": ", ".join(slots),
                })

            except Exception as e:
                print(f"   ! Error processing card {idx}: {e}")

            time.sleep(0.05)

        print("\n[6/6] Saving results to CSV...")
        with open("liverpool.csv", "w", newline="", encoding="utf-8") as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=[
                "restaurant_id", "name", "rating", "reviews",
                "price", "cuisine_location", "timeslots"
            ])
            writer.writeheader()
            writer.writerows(results)

        print("Scraping complete! Saved as restaurants.csv\n")

    finally:
        driver.quit()

    return results


if __name__ == "__main__":
    scrape_opentable_uc_scroll(URL)

[1/6] Creating fresh Chrome options...
[2/6] Launching undetected Chrome...
[3/6] Loading OpenTable page...
[4/6] Scrolling page to load all restaurants...
[5/6] Extracting restaurant data...
Found 50 restaurant cards.

 → Processing restaurant 1/50
 → Processing restaurant 2/50
 → Processing restaurant 3/50
 → Processing restaurant 4/50
 → Processing restaurant 5/50
 → Processing restaurant 6/50
 → Processing restaurant 7/50
 → Processing restaurant 8/50
 → Processing restaurant 9/50
 → Processing restaurant 10/50
 → Processing restaurant 11/50
 → Processing restaurant 12/50
 → Processing restaurant 13/50
 → Processing restaurant 14/50
 → Processing restaurant 15/50
 → Processing restaurant 16/50
 → Processing restaurant 17/50
 → Processing restaurant 18/50
 → Processing restaurant 19/50
 → Processing restaurant 20/50
 → Processing restaurant 21/50
 → Processing restaurant 22/50
 → Processing restaurant 23/50
 → Processing restaurant 24/50
 → Processing restaurant 25/50
 → Processing 